In [3]:
import numpy as np

# Functions

In [558]:
# X: d x n np.ndarray
# return a d x n ndarray
def softmax(X):
    X -= X.max(axis=0)  # for each column, substract the max over row
    eX = np.exp(X) 
    return eX / eX.sum(axis=0)  # softmax for each column

In [559]:
# return a 1xn predicted class label with highest softmax probabilities
# X1: (d+1) x n, features of all samples 
# W1: c x (d+1), weights
def predict_softmax(X1, W1):
    Y_soft = softmax( W1.dot(X1) )  # c x n
    y_hat = np.argmax( Y_soft, axis=0 )  # 1 x n
    return y_hat

In [560]:
# return a LogLoss value between true label y and softmax probabilities
# X: d x n, d features of n samples 
# y: 1 x n, class label in [0, c), dtype='uint8'
# W: c x d, weights
def loss_softmax(X, y, W):
    Y_soft = softmax( W.dot(X) )  # c x n
    y_prob = Y_soft[y, range(len(y))]  # 1 x n
    #print "len(y)=", len(y)
    return -np.log(y_prob).mean()

In [561]:
# Use partial derivative df = (f(X+h) - f(X-h)) / 2h on each feature of X
# func: a function that takes a single argument
# X is the point (numpy array) to evaluate the gradient at
# h: step
def gradient(func, X, h=0.00001):
    grad = np.zeros(X.shape)
    # iterate over all indexes in X
    it = np.nditer(X, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        # evaluate function at x+h
        ix = it.multi_index
        old_value = X[ix]
        X[ix] = old_value + h 
        fxh1 = func(X) # evalute f(x + h)
        #print X, fxh1
        X[ix] = old_value - h 
        fxh2 = func(X) # evalute f(x - h)
        #print X, fxh2
        X[ix] = old_value # restore to previous value (very important!)

        # compute the partial derivative
        grad[ix] = (fxh1 - fxh2) / (2*h) # the slope
        it.iternext() # step to next dimension
    return grad

### Gradient Check

In [562]:
## test 1
f = lambda x: -3*x*x + 2*x + 4
X = np.array(3.0)
gradient( f, X )

array(-16.000000000282455)

In [563]:
# test 2
f = lambda X: X[0]*X[1]
X = np.array([3.0, 4.0])
gradient( f, X )

array([ 4.,  3.])

# Parameters

In [564]:
d = 8 # feature dimension
n = 100 # sample size
c = 5 # num of classes

### Samples and features

In [565]:
np.random.seed(1122)
X = np.random.randn(d, n) # features
vec1 = np.ones(n)
X1 = np.vstack((X, vec1)) # extended features including intercept

### True weights to learn

In [566]:
np.random.seed(1122)
W1 = np.random.randn(c, d+1)   # true weights, including intercept

### True labels

In [567]:
noise = np.random.normal(0, 0.1, (c, n))
Y_soft = softmax( W1.dot(X1) + noise)

In [568]:
y = np.argmax( Y_soft, axis=0 )   # true class labels, in [0, c)

In [570]:
loss_softmax(X1, y, W1)

33.95612339852844

# Training

### Loss function on weights

In [575]:
## given training data (X1 and y), loss is func of W1 only
def loss_on_weights( W1 ):
    return loss_softmax(X1, y, W1)

### Initialize weights

In [586]:
np.random.seed(1122)
W1_hat = np.random.randn(c, d+1) * 0.001

### Gradient Decendent

In [588]:
## Note: this code is generic of loss functions
iter = 0
step = 10 ** -5
while 1:
    loss_new = loss_on_weights( W1_hat )
    grad_W1_hat = gradient( loss_on_weights, W1 )
    W1_hat += - step * grad_W1_hat
    if iter % 100 == 0:
        print "iter %d: loss = %f" % ( iter, loss_new )
    iter += 1;
    if iter >= 1001:
        break

iter 0: loss = 159.797967
iter 100: loss = 158.935277


### Evaluation

In [589]:
(predict(X1, W1) - y)


array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0, -2,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  1,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [590]:
predict(X1, W1_hat) - y

array([-3,  0,  0,  0,  0, -2, -3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  0,  0,  0,  0,  0, -3,  3,  4,  0,  1,  0,  4, -2,  1,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  4,  0,  0,  0, -1,  3,
        1, -2,  0,  1,  1,  0,  0, -2, -2,  0,  1,  1,  0,  0,  0,  3,  2,
        0,  0, -1,  0,  0,  1,  1,  0, -3,  1,  0,  0,  1,  0,  0,  0,  3,
        0, -2,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1])